# Section 4.2.2 in Gordon et al 2023. 
# Last updated: 4/13/23 

In [ ]:
from astroquery.mast import Observations
import numpy as np
import matplotlib.pyplot as plt
import os 

# Obviously because we pulled the real TESS data by manual inspection, this code won't give you totally the same results, but it is the method we used. 

In [ ]:
#TESS observation download: 
obs_table = Observations.query_criteria(obs_collection="TESS", 
                                       sequence_number=14, dataproduct_type='timeseries')
random_indexes = np.random.randint(0, len(obs_table), 100)
print(random_indexes)

from astropy.io import fits
# download random
for i in range(100):
    data_products = Observations.get_product_list(obs_table[i])
    Observations.download_products(data_products, description="Light curves")
  
download_dir = "/Users/lindseygordon/etsfit_local/reliability_testing/mastDownload/" #change to your dir.

# open and plot them all
for root, dirs, files in os.walk(download_dir):
    for name in files:
        fname = root + "/" + name
        data = fits.open(fname, memmap=False)
        t = data[1].data['TIME']
        flux = data[1].data['PDCSAP_FLUX']
        err = data[1].data['PDCSAP_FLUX_ERR']
        id_ = data[0].header["Object"]
        
        fig, ax = plt.subplots(1, figsize=(8,3))
        plt.errorbar(t, flux, err)
        plt.title(id_)
        print(id_)
        plt.show()
        plt.close()
        data.close()     

In [ ]:
interesting = ["TIC 122064388", "TIC 99937762", "TIC 88003319", "TIC 233856183", "TIC 284362766",
              "TIC 198187563", "TIC 115650797", "TIC 99956360", "TIC 26543048", "TIC 302518015", "TIC 286631521"]
flat = ['TIC 158491422', "TIC 288185404", "TIC 154107231", "TIC 137607001", "TIC 406998744",
       "TIC 235930678", "TIC 230073587", "TIC 313304366", "TIC 372758380", "TIC 233283569"]

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(10, 20))

lists_ = [flat, interesting]
for j in range(2):
    use = lists_[j]
    for i in range(10):
        axy = axs[i][j]
        for root, dirs, files in os.walk(download_dir):
            for name in files:
                fname = root + "/" + name
                if use[i] in fname:
                    #print(fname)
                    data = fits.open(fname, memmap=False)
                    t = data[1].data['TIME']
                    flux = data[1].data['PDCSAP_FLUX']
                    err = data[1].data['PDCSAP_FLUX_ERR']
                    id_ = data[0].header["Object"]

                    axy.errorbar(t, flux, err, mfc='black', mec='black', ms=1, fmt=".", ecolor='black')
                    axy.set_title(id_)

                    data.close()
                    
#This is figure YYY in the paper.                   
plt.suptitle("Injected Light Curves")
plt.tight_layout()
plt.savefig("injected_data.png")

In [ ]:
# make csv files of chosen 20 
import pandas as pd
save_dir = "./chosen_data/"
for j in range(2):
    use = lists_[j]
    for i in range(10):
        axy = axs[i][j]
        for root, dirs, files in os.walk(download_dir):
            for name in files:
                fname = root + "/" + name
                if use[i] in fname:
                    #print(fname)
                    data = fits.open(fname, memmap=False)
                    t = data[1].data['TIME']
                    flux = data[1].data['PDCSAP_FLUX']
                    err = data[1].data['PDCSAP_FLUX_ERR']
                    id_ = data[0].header["TICID"]
                    
                    di = {'time': t, 'flux':flux, 'error':err}
                    df = pd.DataFrame(di)
                    df.to_csv(f"{save_dir}data_{id_}.csv")
                    data.close()

# Now run reliability testing: 

In [ ]:
import etsfit.utils.reliability_injection as rel
data_folder = "/Users/lindseygordon/etsfit_local/reliability_testing/chosen_data/"
save_dir = "/Users/lindseygordon/etsfit_local/reliability_testing/output/"
n_injections = 10 # how many fake signals
n_tess = 20 # how many tess lightcurves

# init
ri = rel.reliability_injection(save_dir, n_injections, n_tess, rise=1, fraction=0.6)

# real tess data: 
ri.real_data_load(data_folder)
ri.plot_all_real()

# fake signals:
ri.gen_params()
ri.gen_signals()

# signal injection
ri.gen_injections()
ri.plot_all_injections()

# signal fitting: 
ri.fit_fakes(start=0, n1=5_000, n2=50_000)

# run summary stats: 
ri.retrieve_params()
ri.s_stats()
ri.chi_sq()